In [2]:
import pandas as pd   
import requests 
import os 
import time
from dotenv import load_dotenv

load_dotenv()



True

In [3]:
dataset = pd.read_excel('./prompt_library.xlsx')


In [4]:
dataset.head()

,Prompt
0,Are there any collaborative works between Micr...
1,Assess the potential impact of [Project Name] ...
2,Can we draw any parallels between Microsoft's ...
3,Can we identify any new patterns in the applic...
4,Can you help me understand the technical aspec...


In [5]:
def get_response(data):
    """
    Sends a POST request to the conversation API to obtain a response.

    Parameters:
    - data (dict): Body of the request with prompt data.

    Environment Variables:
    - BASE_URL (required): The Base URL of the API. If not provided, defaults to "http://127.0.0.1:5000".
    
    """
    try:
        base_url = os.environ.get("BASE_URL", "http://127.0.0.1:5000")
        url = f"{base_url}/conversation"
        headers = {
            "content-type": "application/json",
        }

        response = requests.post(url, json=data, headers=headers)

        response.raise_for_status()  # Raises HTTPError for bad responses

        response_json = response.json()

        return response_json

    except requests.exceptions.RequestException as req_exc:
        raise Exception("Error in making the API request.")

    except (KeyError, IndexError) as json_exc:
        print(f"JSON parsing error: {json_exc}")
        raise Exception("Error in parsing the API response.")

In [6]:
# Function to rate the answer
def evaluate_response(response, question):
    """
    Rate the quality of an AI-generated response based on the provided response and original user question.

    Parameters:
    - response (dict): The JSON response obtained from the conversation API, containing AI-generated response.
    - question (str): The original user question.
    """
    evaluation_string = os.environ.get("PROMPT_EVAL_STRING")
    try:
        prompt_data = {
            "messages": [
                {
                    "role": "user",
                    "content": question,
                },
                {
                    "role": "tool",
                    "content": response["choices"][0]["messages"][0]["content"],
                },
                {
                    "role": "assistant",
                    "content": response["choices"][0]["messages"][1]["content"],
                },
                {"role": "user", "content": evaluation_string},
            ]
        }
        response = get_response(prompt_data)
        evaluation = response["choices"][0]["messages"][1]["content"]
        return evaluation

    except (KeyError, IndexError):
        raise Exception("Error in parsing the API response.")

In [7]:
def process_row(row):
    """
    Process a row of data containing a user question and obtain the AI-generated response along with an evaluation response/rating.

    Parameters:
    - row (dict): A dictionary representing a row of data, typically from a DataFrame, containing a 'Prompt' field.

    """
    try:
        question = row['Prompt']
        prompt_data = {"messages": [{"role": "user", "content": question}]}
        start_time = time.time()
        response = get_response(prompt_data)
        end_time = time.time()
        answer  = response["choices"][0]["messages"][1]["content"]
        evaluation = evaluate_response(response=response, question=question)
        response_time = end_time - start_time
        return question, answer, evaluation, response_time
    
    except (KeyError, IndexError):
        raise Exception("Error in parsing the API response.")

In [8]:
# Process each row and store the results in a new DataFrame
results = []
for index, row in dataset.iterrows():
    try:
        if index > 4:
            break
        results.append(process_row(row))
        print(f"Question {index} processed succesfully")
    except Exception as e:
        print(f"Error Question {index} processed unsuccesfully: {e}")
        continue

Question 0 processed succesfully
Question 1 processed succesfully
Request error: 400 Client Error: BAD REQUEST for url: http://127.0.0.1:5000/conversation
Error Question 2 processed unsuccesfully: Error in making the API request.
Question 3 processed succesfully
Question 4 processed succesfully


In [9]:
# Create a new DataFrame from the results
output_dataset = pd.DataFrame(results, columns=['Question', 'Answer', 'Evaluation', 'Response Time (s)'])

In [10]:
# Write the new DataFrame to a new Excel file
output_file_path = 'output_data.xlsx'
output_dataset.to_excel(output_file_path, index=False)

In [11]:
output_dataset

,Question,Answer,Evaluation,Response Time (s)
0,Are there any collaborative works between Micr...,"Yes, there are collaborative works between Mic...","I'm sorry, but as an AI language model, I don'...",39.451646
1,Assess the potential impact of [Project Name] ...,The requested information is not available in ...,"As an AI language model, I don't have the capa...",19.418135
2,Can we identify any new patterns in the applic...,The requested information is not available in ...,"As an AI language model, I am not capable of s...",21.586031
3,Can you help me understand the technical aspec...,The requested information is not available in ...,"As an AI language model, I don't have the capa...",15.216946


In [12]:
print(f"Processing complete. Results saved to {output_file_path}")

Processing complete. Results saved to output_data.xlsx
